In [66]:
from dotenv import load_dotenv
load_dotenv()
import os
from langchain_google_genai import ChatGoogleGenerativeAI

google_api_key=os.environ['Gemini_API_key']


In [67]:
model=ChatGoogleGenerativeAI(model='gemini-1.5-flash',google_api_key=google_api_key)
output=model.invoke("hi")
print(output.content)

Hi there! How can I help you today?


In [68]:
from langgraph.types import Command
from langgraph.prebuilt import create_react_agent


In [69]:
def add_number(state):
    result=state["num1"]+state["num2"]
    print(f"addition is {result}")
    return Command(goto="multiply",update={"sum":result})
#goto="multiply" tells LangGraph what node to execute next
#update={"sum": result} updates the shared state by adding a new key "sum".



In [70]:
state={"num1":10,"num2":20}
add_number(state)


addition is 30


Command(update={'sum': 30}, goto='multiply')

In [71]:
state

{'num1': 10, 'num2': 20}


## Creating one dummy multiagent
it is for network/collab multiagent

In [72]:
from langchain_core.tools import tool


In [73]:

@tool
def transfer_to_multiplication_expert():
    """Ask multiplication agent for help"""
    return

In [74]:

@tool
def transfer_to_addition_expert():
    """Ask addition agent for help"""
    return

In [75]:
llm_with_tool=model.bind_tools([transfer_to_addition_expert])

In [76]:
response=llm_with_tool.invoke("hi")


In [77]:
response.content


'Hello! How can I help you today?'

In [78]:
response.tool_calls


[]

In [79]:
response=llm_with_tool.invoke("what is 2+2?")


In [80]:
response.content


'I am sorry, I cannot answer this question. I do not have the functionality to perform calculations.'

In [81]:
response.tool_calls


[]

In [82]:

system_prompt = (
        "You are an addition expert, you can ask the multiplication expert for help with multiplication."
        "Always do your portion of calculation before the handoff."
    )

In [83]:
messages = [{"role": "system", "content": system_prompt}] + ["can you tell me the addition of 2 and 2?"]


In [84]:
messages


[{'role': 'system',
  'content': 'You are an addition expert, you can ask the multiplication expert for help with multiplication.Always do your portion of calculation before the handoff.'},
 'can you tell me the addition of 2 and 2?']

In [85]:
from typing_extensions import Literal
from langgraph.graph import MessagesState,StateGraph, START,END

In [86]:
def additional_expert(state:MessagesState)-> Command[Literal["multiplication_expert", "__end__"]]:
    
    system_prompt = (
        "You are an addition expert, you can ask the multiplication expert for help with multiplication."
        "Always do your portion of calculation before the handoff."
    )
    
    messages = [{"role": "system", "content": system_prompt}] + state["messages"]
    
    
    ai_msg = model.bind_tools([transfer_to_multiplication_expert]).invoke(messages)
    
    
    if len(ai_msg.tool_calls) > 0:
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred",
            "tool_call_id": tool_call_id,
        }
        
        return Command(
            goto="multiplication_expert", update={"messages": [ai_msg, tool_msg]}
        )
    return {"messages": [ai_msg]}

In [87]:

##Agent2
def multiplication_expert(state:MessagesState)-> Command[Literal["additional_expert", "__end__"]]:
    
    system_prompt = (
        "You are a multiplication expert, you can ask an addition expert for help with addition. "
        "Always do your portion of calculation before the handoff."
    )
    
    messages = [{"role": "system", "content": system_prompt}] + state["messages"]
    
    ai_msg = model.bind_tools([transfer_to_addition_expert]).invoke(messages)
    
    if len(ai_msg.tool_calls) > 0:
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred",
            "tool_call_id": tool_call_id,
        }
        return Command(goto="additional_expert", update={"messages": [ai_msg, tool_msg]})
    return {"messages": [ai_msg]}

In [88]:
graph=StateGraph(MessagesState)


In [89]:
graph.add_node("additional_expert",additional_expert)
graph.add_node("multiplication_expert",multiplication_expert)

In [90]:
graph.add_edge(START, "additional_expert")


In [91]:
app=graph.compile()


In [92]:
result=app.invoke({"messages":[("user","what's (3 + 5) * 12. Provide me the output")]})
result

{'messages': [HumanMessage(content="what's (3 + 5) * 12. Provide me the output", additional_kwargs={}, response_metadata={}, id='b1f00657-4880-4df0-8278-b4fd4925b17d'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'MALFORMED_FUNCTION_CALL', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--d12590d6-773c-44fa-9e6c-eb1e2964c1b1-0', usage_metadata={'input_tokens': 58, 'output_tokens': 0, 'total_tokens': 58, 'input_token_details': {'cache_read': 0}})]}

## With realtime tool
